# BioBERT für RE

In [1]:
import sys
sys.path.append("../")

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"  # -1 to use the cpu
import torch
torch.cuda.device_count()

1

In [3]:
from tlbiore.models import ppi_sequence_classifier, utils
from pytorch_lightning import Trainer

MIN_EPOCHS = 2
MAX_EPOCHS = 4

Using TensorFlow backend.


In [5]:
bert_finetuner = ppi_sequence_classifier.BertBiomedicalRE()

# most basic trainer, uses good defaults (1 gpu)
trainer = pl.Trainer(gpus=1, max_nb_epochs=MAX_EPOCHS, min_nb_epochs=MIN_EPOCHS)    
trainer.fit(bert_finetuner) 

OSError: Model name '/content/drive/My Drive/TransferLearning/biobert_v1.1._pubmed' was not found in model name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-japanese, bert-base-japanese-whole-word-masking, bert-base-japanese-char, bert-base-japanese-char-whole-word-masking, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1). We assumed 'https://s3.amazonaws.com/models.huggingface.co/bert//content/drive/My Drive/TransferLearning/biobert_v1.1._pubmed/config.json' was a path or url to a configuration file named config.json or a directory containing such a file b

In [ ]:
# view tensorboard logs 
logging.info(f'View tensorboard logs by running\ntensorboard --logdir {os.getcwd()}')
logging.info('and going to http://localhost:6006 on your browser')

In [ ]:
trainer.test()